<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/Andrey/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [79]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
Using competition: 26jun23
  teamId  teamName                       submissionDate       score    
--------  -----------------------------  -------------------  -------  
10612706  Michael Kondrashov             2023-06-30 11:34:09  0.88424  
10612701  Никита Угрюмов                 2023-06-30 11:59:02  0.88402  
10612695  Ne1tr0N [Maxim Mashtaler]      2023-06-29 22:06:01  0.88242  
10614699  Dmitriy Geyzha                 2023-06-30 11:59:52  0.87769  
10616508  Ulyana Klushova                2023-06-30 12:24:49  0.87710  
10614521  Konstantin Zeigman             2023-06-30 11:37:17  0.87691  
10612698  Бобы                           2023-06-30 10:43:59  0.87539  
10617156  Team 5                         2023-06-30 11:08:02  0.87355  
10612681  Maria Alyushina                2023-06-29 18:43:03  0.87097  
10612680  esheshka                       2023-06-30 12:29:16  0.87067  
10613713  Mikha

In [81]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 1.49 s, sys: 76 ms, total: 1.56 s
Wall time: 50.5 s


In [82]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC

from sklearn.neural_network import MLPClassifier as MLPC

from sklearn.model_selection import GridSearchCV as GSCV

In [83]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc')

CPU times: user 5.15 ms, sys: 0 ns, total: 5.15 ms
Wall time: 26.3 ms


In [84]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc');

CPU times: user 4.22 ms, sys: 0 ns, total: 4.22 ms
Wall time: 4.24 ms


In [85]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid')
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')

CPU times: user 6.26 ms, sys: 12 µs, total: 6.27 ms
Wall time: 17.3 ms


In [86]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm'])

CPU times: user 9.43 s, sys: 959 ms, total: 10.4 s
Wall time: 37 s


In [87]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 4)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


⏳ started. You have 120 sec. Good luck!


# **Imports**

In [88]:
!pip install lightgbm
!pip install catboost

from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


# **Interesting**

In [90]:
df_test = dfTrx.join(gender, on='cid', how='inner')
df_test = df_test[~df_test.gender.isna()]

In [91]:
#@title MCC

mcc_list = list(set(df_test.mcc))
df_mcc = pd.DataFrame({'mcc': mcc_list})

zero = df_test[df_test.gender == 0][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
zero = df_mcc.join(pd.DataFrame({'zero': zero}), on='mcc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
one = df_mcc.join(pd.DataFrame({'one': one}), on='mcc', how='outer').fillna(0)

df_mcc = df_mcc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='mcc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='mcc', how='left')

interesting_mcc = set(df_mcc[(df_mcc['count'] > 100) & ((df_mcc['%'] > 70) | (df_mcc['%'] < 30))].mcc)

interesting_mcc

{62, 63, 70, 71, 82, 106, 149, 150, 161}

In [92]:
#@title TTC

ttc_list = list(set(df_test.ttc))
df_ttc = pd.DataFrame({'ttc': ttc_list})

zero = df_test[df_test.gender == 0][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
zero = df_ttc.join(pd.DataFrame({'zero': zero}), on='ttc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'ttc']].drop_duplicates().groupby('ttc').agg('size').fillna(0)
one = df_ttc.join(pd.DataFrame({'one': one}), on='ttc', how='outer').fillna(0)

df_ttc = df_ttc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='ttc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='ttc', how='left')

interesting_ttc = set(df_ttc[(df_ttc['count'] > 100) & ((df_ttc['%'] > 70) | (df_ttc['%'] < 30))].ttc)

interesting_ttc

set()

In [93]:
#@title TID

tid_list = list(set(df_test.tid))
df_tid = pd.DataFrame({'tid': tid_list})

zero = df_test[df_test.gender == 0][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
zero = df_tid.join(pd.DataFrame({'gender': zero}), on='tid', how='outer').fillna(0).astype({'gender': int})
one = df_test[df_test.gender == 1][['cid', 'tid']].drop_duplicates().groupby('tid').agg('size').fillna(0)
one = df_tid.join(pd.DataFrame({'gender': one}), on='tid', how='outer').fillna(0).astype({'gender': int})

df_tid = df_tid
df_tid = df_tid.join(zero[['gender']] + one[['gender']]).rename(columns={'gender': 'count'}).join(100 * zero[['gender']] / (zero[['gender']] + one[['gender']])).rename(columns={'gender': '%'})

interesting_tid = set(df_tid[(df_tid['count'] > 100) & ((df_tid['%'] > 70) | (df_tid['%'] < 30))].tid)

interesting_tid

{'10244893', '20001004', '20001033', '20828301', '940047'}

# **My Code**

In [94]:
#@title Настройки

large_count = 2000
large_sum = -1000000000

k = 1
k1 = 0.8
k2 = 0.85
# k3 = 1.25
k3 = 8

In [95]:
#@title Определение наиболее используемых mcc
%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > large_count * k1].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < large_sum * k1].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 1.1 s, sys: 66.7 ms, total: 1.17 s
Wall time: 4.39 s


36

In [96]:
#@title Определение наиболее используемых ttc
%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > large_count * k2].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < large_sum * k2) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 997 ms, sys: 43.6 ms, total: 1.04 s
Wall time: 4.06 s


25

In [97]:
#@title Определение наиболее используемых tid
%%time
tid_by_cid = dfTrx[['cid', 'tid']].drop_duplicates().astype({'tid': str}).groupby('cid').agg({'tid': ' '.join}).sort_values('cid')

large_tid_count = dfTrx[['cid', 'tid']].drop_duplicates().groupby('tid').agg({'cid': 'size'}).sort_values(by = 'cid')
large_tid_sum = dfTrx[['amt', 'tid']].groupby('tid').agg('sum').sort_values(by = 'amt')

most_popular_tid = set(large_tid_count[large_tid_count.cid > large_count * k3].index.to_list()) | set(large_tid_sum[(large_tid_sum.amt < large_sum * k3) | (large_tid_sum.amt > 1000000000)].index.to_list())
len(most_popular_tid)

CPU times: user 6.24 s, sys: 150 ms, total: 6.39 s
Wall time: 19.1 s


0

# **Начало создания df по cid**

In [98]:
#@title Сумма

def s(to_df):
    return to_df / to_df.sum(axis=0)
    # return to_df / np.sqrt((to_df**2).sum(axis=0))
    # return to_df / (np.sqrt(to_df**2)).sum(axis=0)

In [99]:
#@title Настройки

stat = ['count', 'sum', 'median']

In [100]:
%%time
df = df_.copy(deep=True)

CPU times: user 251 µs, sys: 11 µs, total: 262 µs
Wall time: 182 µs


In [101]:
#@title MCC
%%time
to_df = df_.copy(deep=True)

for mcc_index in (most_popular_mcc - interesting_mcc):
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

stat_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc or interesting_mcc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_mcc.columns += '_mcc'
to_df = to_df.join(stat_by_other_mcc, how='left')

to_df = s(to_df) * 75
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for mcc_index in interesting_mcc:
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

to_df = s(to_df) * 200
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 2.24 s, sys: 25.6 ms, total: 2.26 s
Wall time: 7.96 s


In [102]:
#@title TTC
to_df = df_.copy(deep=True)

for ttc_index in (most_popular_ttc - interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

stat_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc or interesting_ttc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_ttc.columns += '_ttc'
to_df = to_df.join(stat_by_other_ttc, how='left')

to_df = s(to_df) * 1/2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for ttc_index in (interesting_ttc):
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [103]:
#@title AMT

to_df = df_.copy(deep=True)

to_df = dfTrx[['cid', 'amt']].groupby('cid').agg(stat).add_prefix('').droplevel(0, axis='columns')
to_df = df_.join(to_df, how='left').fillna(0)

to_df = s(to_df) * 2/3
df = df.join(to_df, on = 'cid', how='left')

In [104]:
#@title Time
%%time
to_df = df_.copy(deep=True)

for hour in range(0, 1):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df) * 2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(1, 9):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

# to_df = df_.copy(deep=True)
# for hour in range(9, 24):
#     stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
#     stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
#     to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

# to_df = s(to_df)
# df = df.join(to_df, on = 'cid', how='left')

CPU times: user 3.3 s, sys: 20.9 ms, total: 3.32 s
Wall time: 3.31 s


In [105]:
#@title Date
%%time
to_df = df_.copy(deep=True)

# for weekday in [0, 3, 4, 5, 6]:
#     stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
#     stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
#     to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for weekday in [1, 2]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 328 ms, sys: 1.03 ms, total: 329 ms
Wall time: 339 ms


In [106]:
#@title TID
to_df = df_.copy(deep=True)

for tid_index in (most_popular_tid - interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

stat_by_other_tid = dfTrx[~dfTrx.tid.isin(most_popular_tid or interesting_tid)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_tid.columns += '_tid'
to_df = to_df.join(stat_by_other_tid, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

to_df = df_.copy(deep=True)

for tid_index in (interesting_tid):
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

to_df = s(to_df) * 50
df = df.join(to_df, on = 'cid', how='left')

In [107]:
df = df.fillna(0).astype('float')
df.columns = [str(c) for c in df.columns]

In [108]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.4, random_state=0) # split train observation into local train/test sets (to score performance)


CPU times: user 25.7 ms, sys: 0 ns, total: 25.7 ms
Wall time: 26.4 ms


# **Обучение реальное**

In [111]:
model_final = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)

model1 = MLPC(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)
model2 = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model3 = CBC(iterations=200)
model4 = LGBMC()

clf1 = model1.fit(trainX, trainY.gender)
clf2 = model2.fit(trainX, trainY.gender)
clf3 = model3.fit(trainX, trainY.gender)
clf4 = model4.fit(trainX, trainY.gender)

pY1_r = clf1.predict_proba(testX)[:, 1]
pY2_r = clf2.predict_proba(testX)[:, 1]
pY3_r = clf3.predict_proba(testX)[:, 1]
pY4_r = clf4.predict_proba(testX)[:, 1]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Learning rate set to 0.09957
0:	learn: 0.6656943	total: 144ms	remaining: 28.7s
1:	learn: 0.6445882	total: 236ms	remaining: 23.4s
2:	learn: 0.6253448	total: 323ms	remaining: 21.2s
3:	learn: 0.6076534	total: 440ms	remaining: 21.5s
4:	learn: 0.5972474	total: 543ms	remaining: 21.2s
5:	learn: 0.5864506	total: 660ms	remaining: 21.3s
6:	learn: 0.5747662	total: 766ms	remaining: 21.1s
7:	learn: 0.5631818	total: 884ms	remaining: 21.2s
8:	learn: 0.5550646	total: 982ms	remaining: 20.8s
9:	learn: 0.5464971	total: 1.09s	remaining: 20.8s
10:	learn: 0.5388928	total: 1.21s	remaining: 20.7s
11:	learn: 0.5334489	total: 1.32s	remaining: 20.8s
12:	learn: 0.5276814	total: 1.44s	remaining: 20.7s
13:	learn: 0.5227316	total: 1.55s	remaining: 20.6s
14:	learn: 0.5176091	total: 1.64s	remaining: 20.3s
15:	learn: 0.5125028	total: 1.76s	remaining: 20.3s
16:	learn: 0.5082158	total: 1.87s	remaining: 20.1s
17:	learn: 0.5024984	total: 1.96s	remaining: 19.8s
18:	learn: 0.4989825	total: 2.06s	remaining: 19.6s
19:	learn: 0

In [112]:
pY = (pY1_r + 2*pY2_r + 4*pY3_r + 2*pY4_r) / 9

# print('Local AUC ROC', roc_auc_score(testY.gender, pY))
print(pY.sum() / pY.shape[0])

0.4428708344165044


In [116]:
pY = pd.DataFrame(pY, columns=['gender'])
pY.index.name = 'cid'
print(pY.sum() / pY.shape[0]), pY

gender    0.442871
dtype: float64


(None,
         gender
 cid           
 0     0.533530
 1     0.278108
 ...        ...
 1998  0.637415
 1999  0.948957
 
 [2000 rows x 1 columns])

In [114]:
pY.reset_index().to_csv('💳Baseline.csv', index_label='cid', index=False, float_format='%10f')

In [117]:
tmr.ShowTime()

Runtime is 925 sec > 120 sec limit!!!
